### Tidy the GEFCom2012 dataset
From 3 file, load_raw.csv, n034_ensemble.csv and temp.csv create the complete.csv file. Use value from ensemble prediction to impute missing value in load_raw, and add temperature values from temp.csv to the final complete dataframe.

#### Read data from csv file

In [1]:
library(tidyr)
library(dplyr)
library(lubridate)

loadRawFile = "GEFCom2012/load_raw.csv"
tempRawFile = "GEFCom2012/temp.csv"
ensemblePredictionFile = "GEFCom2012/n034_ensemble.csv"

#Define a class num.with.commas to probably transform string with comma to number
setClass("num.with.commas")
setAs("character", "num.with.commas", 
        function(from) as.numeric(gsub(",", "", from) ) )

loadRawClasses = c('factor', rep("numeric", 3), rep("num.with.commas", 24))

df = read.csv(loadRawFile, stringsAsFactors=FALSE, colClasses=loadRawClasses)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



#### Tidy loadraw GEFCom2012 data

In [2]:
tidyLoadDf = df %>%  gather(Hour, Consumption, h1:h24) %>%
                 mutate(Hour=as.numeric(substr(Hour, 2, nchar(Hour)))-1) %>%
                 unite(Date, year, month, day, sep='-') %>%
                 unite(DateTime, Date, Hour, sep=' ') %>%
                 mutate(DateTime=as.POSIXct(strptime(DateTime, "%Y-%m-%d %H", tz = "GMT"))) %>%
                 arrange(DateTime, zone_id)


In [3]:
#Spead consumption for each zone into seperate columns, so that we can easily select consumption for each zone later
tidyLoadDf = tidyLoadDf %>% mutate(zone_id=paste0('zone.',as.character(zone_id))) %>%
                      spread(zone_id, Consumption, fill = NA, convert = FALSE)
#Remove 2008/June data, because it's not complete, Create Zone 21 as sum of 20 zones consumption
tidyLoadDf = tidyLoadDf %>% filter(DateTime<as.POSIXct(strptime("2008-06-01 00:00:00", "%Y-%m-%d %H:%M:%S")))
#Add total consumption collumn
tidyLoadDf %>% select(zone.1:zone.20) %>% rowSums(na.rm=TRUE) -> tidyLoadDf$total

#### Tidy ensemble prediction GEFCom2012 data

In [4]:
ensembleClasses = c('NULL', 'factor', rep("numeric", 27))
df = read.csv(ensemblePredictionFile, stringsAsFactors=FALSE, colClasses=ensembleClasses)
tidyEnsembleDf = df %>%  gather(Hour, Consumption, h1:h24) %>%
                 mutate(Hour=as.numeric(substr(Hour, 2, nchar(Hour)))-1) %>%
                 unite(Date, year, month, day, sep='-') %>%
                 unite(DateTime, Date, Hour, sep=' ') %>%
                 mutate(DateTime=as.POSIXct(strptime(DateTime, "%Y-%m-%d %H", tz = "GMT"))) %>%
                 arrange(DateTime) %>%
                 mutate(zone_id=paste0('zone.',as.character(zone_id))) %>%
                 spread(zone_id, Consumption, fill = NA, convert = FALSE) %>% #Remove forecast, only need backcast
                 filter(DateTime<as.POSIXct(strptime("2008-06-01 00:00:00", "%Y-%m-%d %H:%M:%S"))) 

#### Merge ensemble prediction to raw load data, and add temperature data

In [5]:
fullDf = tidyLoadDf
fullDf[is.na(fullDf$zone.1), ] = tidyEnsembleDf

In [7]:
tempClasses = c('POSIXct', rep("numeric", 11))
tempDf = read.csv(tempRawFile, stringsAsFactors=FALSE, colClasses=tempClasses)
tempDf = tempDf %>% mutate(DateTime = DateTime - minutes(30))
completeDf = left_join(fullDf, tempDf, by="DateTime")
#Check if there is any not complete case
completeDf %>% filter(!complete.cases(.))

In [13]:
write.csv(completeDf, "GEFCom2012/complete.csv")

In [9]:
completeDf %>% filter(is.na(T01))